In [1]:
import ee
import os
import numpy as np
import pandas as pd
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'
ee.Initialize()

In [2]:
df = pd.read_csv("centers_reduce.csv",index_col = "Unnamed: 0")
df.head()

,NAMEASCII,POP_MAX,SOV0NAME,wof_id
0,Tokyo,35676000,Japan,102031307
1,New York,19040000,United States,85977539
2,Mexico City,19028000,Mexico,857683023
3,Mumbai,18978000,India,102030609
4,Sao Paulo,18845000,Brazil,1141909435


In [3]:
# for k in range(318,1000):
#     city_name = df.iloc[k,0]
#     coun_name = df.iloc[k,2]
#     fid = int(df.iloc[k,3])
#     center = ee.FeatureCollection("users/zhouzz400/Boundries/city_center").filter(ee.Filter.eq("wof_id",fid)).geometry()
#     region = center.buffer(31000)
#     GAIA = ee.ImageCollection("users/zhouzz400/GAIA").filterBounds(region).mosaic()
#     dis_list = ee.List.sequence(1000,30000,1000)
    
#     resU = []
#     yDic = [34,29,24,19,14,9,4,1]
#     for yIndex in yDic:
#         yIndex = ee.Number(yIndex)
#         GAIA_year = GAIA.gte(yIndex)
#         def getUrban(dis):
#             buffer = center.buffer(dis)
#             buffer_urban = GAIA_year.eq(1).clip(buffer)
#             area_imag = buffer_urban.multiply(ee.Image.pixelArea());
#             sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") ).round()
#             return sumarea
#         areaA_urban = dis_list.map(getUrban)
#         resU.append( areaA_urban.getInfo())
    
#     resW = []
#     yDic2 = [1985,1990,1995,2000,2005,2010,2015,2018]
#     for yIndex in yDic2:
#         year = ee.Number(yIndex)
#         water = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq("year",year)).first().neq(1)
#         #dis_list = ee.List.sequence(1000,30000,1000)
#         def getWater(dis):
#             buffer = center.buffer(dis);
#             buffer_urban = water.eq(1).clip(buffer)
#             area_imag = buffer_urban.multiply(ee.Image.pixelArea())
#             sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("waterClass") ).round()
#             return sumarea
#         areaA_water = dis_list.map(getWater)
#         resW.append(areaA_water.getInfo())
     
#     yDic = ee.List([34,29,24,19,14,9,4,1])
#     def totoalL(yIndex):
#         #dis_list = ee.List.sequence(1000,30000,1000)
#         def getTotal(dis):
#             return center.buffer(dis).area()
#         areaA_total = dis_list.map(getTotal)
#         return areaA_total
#     resT= yDic.map(totoalL).getInfo()
    
#     cols = ["Distance","Total","1985_U","1990_U","1995_U","2000_U","2005_U","2010_U","2015_U","2018_U","1985_W","1990_W","1995_W","2000_W","2005_W","2010_W","2015_W","2018_W","1985_D","1990_D","1995_D","2000_D","2005_D","2010_D","2015_D","2018_D"]
#     df_r = pd.DataFrame(columns=cols)
#     df_r.iloc[:,0]=[(x+1)*1000 for x in range(30)]
#     df_r.iloc[:,1]=interSubt(resT[0])
#     for i in range(8):
#         df_r.iloc[:,i+2]=interSubt(resU[i])
#         df_r.iloc[:,i+10]=interSubt(resW[i])
#         for j in range(30):
#             df_r.iloc[j,i+18] = df_r.iloc[j,i+2]/(df_r.iloc[j,1]-df_r.iloc[j,i+10])
#     name ="./RES/"+str(k)+"_"+coun_name+"_"+ city_name+".csv"
#     df_r.to_csv(name)

In [4]:
def interSubt(list1):
    list2 = [0] +list1[:-1]
    delta = [list1[i]-list2[i] for i in range(0,len(list1))]
    return delta;

In [5]:
def revS(k):
    city_name = df.iloc[k,0]
    coun_name = df.iloc[k,2]
    fid = int(df.iloc[k,3])
    center = ee.FeatureCollection("users/zhouzz400/Boundries/city_center").filter(ee.Filter.eq("wof_id",fid)).geometry()
    region = center.buffer(31000)
    GAIA = ee.ImageCollection("users/zhouzz400/GAIA").filterBounds(region).mosaic()
    dis_list = ee.List.sequence(1000,30000,1000)
    
    resU = []
    yDic = [34,29,24,19,14,9,4,1]
    for yIndex in yDic:
        yIndex = ee.Number(yIndex)
        GAIA_year = GAIA.gte(yIndex)
        def getUrban(dis):
            buffer = center.buffer(dis)
            buffer_urban = GAIA_year.eq(1).clip(buffer)
            area_imag = buffer_urban.multiply(ee.Image.pixelArea());
            sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("b1") ).round()
            return sumarea
        areaA_urban = dis_list.map(getUrban)
        resU.append( areaA_urban.getInfo())
    
    resW = []
    yDic2 = [1985,1990,1995,2000,2005,2010,2015,2018]
    for yIndex in yDic2:
        year = ee.Number(yIndex)
        water = ee.ImageCollection("JRC/GSW1_1/YearlyHistory").filter(ee.Filter.eq("year",year)).first().neq(1)
        #dis_list = ee.List.sequence(1000,30000,1000)
        def getWater(dis):
            buffer = center.buffer(dis);
            buffer_urban = water.eq(1).clip(buffer)
            area_imag = buffer_urban.multiply(ee.Image.pixelArea())
            sumarea = ee.Number(area_imag.reduceRegion(**{"reducer": ee.Reducer.sum(),"scale": 30,"maxPixels": 1e9}).get("waterClass") ).round()
            return sumarea
        areaA_water = dis_list.map(getWater)
        resW.append(areaA_water.getInfo())
     
    yDic = ee.List([34,29,24,19,14,9,4,1])
    def totoalL(yIndex):
        #dis_list = ee.List.sequence(1000,30000,1000)
        def getTotal(dis):
            return center.buffer(dis).area()
        areaA_total = dis_list.map(getTotal)
        return areaA_total
    resT= yDic.map(totoalL).getInfo()
    
    cols = ["Distance","Total","1985_U","1990_U","1995_U","2000_U","2005_U","2010_U","2015_U","2018_U","1985_W","1990_W","1995_W","2000_W","2005_W","2010_W","2015_W","2018_W","1985_D","1990_D","1995_D","2000_D","2005_D","2010_D","2015_D","2018_D"]
    df_r = pd.DataFrame(columns=cols)
    df_r.iloc[:,0]=[(x+1)*1000 for x in range(30)]
    df_r.iloc[:,1]=interSubt(resT[0])
    for i in range(8):
        df_r.iloc[:,i+2]=interSubt(resU[i])
        df_r.iloc[:,i+10]=interSubt(resW[i])
        for j in range(30):
            df_r.iloc[j,i+18] = df_r.iloc[j,i+2]/(df_r.iloc[j,1]-df_r.iloc[j,i+10])
    name ="./RES/"+str(k)+"_"+coun_name+"_"+ city_name+".csv"
    df_r.to_csv(name)
    return 1

In [ ]:
for k in range(1111,2000):
    try:
        a = revS(k)
    except ConnectionAbortedError:
        import ee
        import os
        os.environ['HTTP_PROXY'] = 'http://127.0.0.1:10809'
        os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:10809'
        ee.Initialize()
        a = revS(k)
        import datetime
        time = datetime.datetime.now().strftime('%Y.%m.%d-%H:%M:%S') 
        print(k," time: "+ time)

1111  time: 2020.02.12-01:18:48
1130  time: 2020.02.12-02:14:32
1160  time: 2020.02.12-03:08:23
1193  time: 2020.02.12-04:03:45
1229  time: 2020.02.12-04:58:26
1264  time: 2020.02.12-05:53:31
1298  time: 2020.02.12-06:49:14
1334  time: 2020.02.12-07:43:54


In [ ]:
## 710